En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [3]:
file_path = "../../farmers-protest-tweets-2021-2-4.json"

# Importar librerias

In [98]:
import pandas as pd
from collections import Counter
import emoji
from typing import List, Tuple
import datetime
from memory_profiler import profile
import regex

# Leer archivo

In [87]:
df = pd.read_json(file_path, lines=True)
df

,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,retweetCount,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/ArjunSinghPanam/status/136...,2021-02-24 09:23:35+00:00,The world progresses while the Indian police a...,The world progresses while the Indian police a...,1364506249291784198,"{'username': 'ArjunSinghPanam', 'displayname':...",[https://twitter.com/ravisinghka/status/136415...,[https://t.co/es3kn0IQAF],0,0,...,0,1364506249291784198,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,NaN,{'url': 'https://twitter.com/RaviSinghKA/statu...,"[{'username': 'narendramodi', 'displayname': '..."
1,https://twitter.com/PrdeepNain/status/13645062...,2021-02-24 09:23:32+00:00,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,1364506237451313155,"{'username': 'PrdeepNain', 'displayname': 'Pra...",[],[],0,0,...,0,1364506237451313155,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,NaN,None,"[{'username': 'Kisanektamorcha', 'displayname'..."
2,https://twitter.com/parmarmaninder/status/1364...,2021-02-24 09:23:22+00:00,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,ਪੈਟਰੋਲ ਦੀਆਂ ਕੀਮਤਾਂ ਨੂੰ ਮੱਦੇਨਜ਼ਰ ਰੱਖਦੇ ਹੋਏ \nਮੇ...,1364506195453767680,"{'username': 'parmarmaninder', 'displayname': ...",[],[],0,0,...,0,1364506195453767680,pa,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,NaN,None,None
3,https://twitter.com/anmoldhaliwal/status/13645...,2021-02-24 09:23:16+00:00,@ReallySwara @rohini_sgh watch full video here...,@ReallySwara @rohini_sgh watch full video here...,1364506167226032128,"{'username': 'anmoldhaliwal', 'displayname': '...",[https://youtu.be/-bUKumwq-J8],[https://t.co/wBPNdJdB0n],0,0,...,0,1364350947099484160,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,NaN,None,"[{'username': 'ReallySwara', 'displayname': 'S..."
4,https://twitter.com/KotiaPreet/status/13645061...,2021-02-24 09:23:10+00:00,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,#KisanEktaMorcha #FarmersProtest #NoFarmersNoF...,1364506144002088963,"{'username': 'KotiaPreet', 'displayname': 'Pre...",[],[],0,0,...,0,1364506144002088963,und,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117402,https://twitter.com/rickyrickstir/status/13600...,2021-02-12 01:37:02+00:00,#FarmersProtest #KisanAndolan #KisaanMajdoorEk...,#FarmersProtest #KisanAndolan #KisaanMajdoorEk...,1360040182771163138,"{'username': 'rickyrickstir', 'displayname': '...",[],[],0,0,...,0,1360040182771163138,und,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,NaN,None,None
117403,https://twitter.com/PunjabTak/status/136004014...,2021-02-12 01:36:53+00:00,PM मोदी की अपील के बीच संयुक्त किसान मोर्चा का...,PM मोदी की अपील के बीच संयुक्त किसान मोर्चा का...,1360040146402373637,"{'username': 'PunjabTak', 'displayname': 'Punj...",[https://youtu.be/aG3qHGwoYag],[https://t.co/AzZNOGI8BX],0,0,...,0,1360040146402373637,hi,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,None,None
117404,https://twitter.com/ish_kayy/status/1360040134...,2021-02-12 01:36:50+00:00,United we stand.\nDivided we fall\n#Mahapancha...,United we stand.\nDivided we fall\n#Mahapancha...,1360040134230556678,"{'username': 'ish_kayy', 'displayname': 'ishy'...",[],[],0,65,.

## Q1

Se extrae el nombre de usuario de la columna user, y después se ocupa la función Counter() para poder contar las 10 fechas que más se repiten, y el usuario que más se repite en cada una de esas fechas.

Para un ahorro de memoria, se filtra solo las columnas que se necesitan al comienzo para no ocupar todo el df

In [94]:
@profile
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    df = pd.read_json(file_path, lines=True)
    df = df[['date','user']]

    # Extraer el usuario de la columna 
    df['username'] = df['user'].apply(lambda x: x.get('username') if isinstance(x, dict) and 'username' in x else None)

    # Pasar a fecha para que no influya la hora
    df['date'] = pd.to_datetime(df['date']).dt.date

    # Contador de usuarios
    username_counts = Counter(df[['date', 'username']].apply(tuple, axis=1))

    # Top 10 de fechas
    top_10 = Counter(df['date']).most_common(10)

    lista = list()

    # Ir viendo top usuario por fecha
    for date, count in top_10:
        date_data = df[df['date'] == date]
        username_counts_for_date = Counter(date_data['username'])
        
        # Encontrar el usuario más frecuente en la fecha
        most_frequent_username = username_counts_for_date.most_common(1)[0][0]
        lista.append((date, most_frequent_username))
    return lista

print(q1_memory("../../farmers-protest-tweets-2021-2-4.json"))

ERROR: Could not find file /var/folders/qh/33532dns5z5561hpfw82qfz00000gp/T/ipykernel_38184/1197558600.py
[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


In [96]:
%reload_ext memory_profiler
%mprun -f q1_memory q1_memory(file_path)

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
ERROR: Could not find file /var/folders/qh/33532dns5z5561hpfw82qfz00000gp/T/ipykernel_38184/1197558600.py



Filename: /usr/local/lib/python3.11/site-packages/memory_profiler.py

Line #    Mem usage    Increment  Occurrences   Line Contents
  1185   5232.3 MiB   5232.3 MiB           1               @wraps(wrapped=func)
  1186                                                     def wrapper(*args, **kwargs):
  1187   5232.3 MiB      0.0 MiB           1                   prof = get_prof()
  1188   3896.7 MiB  -1335.6 MiB           1                   val = prof(func)(*args, **kwargs)
  1189   3896.8 MiB      0.0 MiB           1                   show_results_bound(prof)
  1190   3896.8 MiB      0.0 MiB           1                   return val

In [97]:
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    df = pd.read_json(file_path, lines=True)
        
    df['username'] = df['user'].apply(lambda x: x.get('username') if isinstance(x, dict) and 'username' in x else None)

    df['date'] = pd.to_datetime(df['date']).dt.date
    conteo = Counter(df['date'])
    username_counts = Counter(df[['date', 'username']].apply(tuple, axis=1))
    top_10 = conteo.most_common(10)

    lista = list()

    for date, count in top_10:
        date_data = df[df['date'] == date]
        username_counts_for_date = Counter(date_data['username'])
        most_frequent_username = username_counts_for_date.most_common(1)[0][0]
        lista.append((date, most_frequent_username))

    return lista
q1_time(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

## Q2

Para esta pregunta se usa la librería emoji para poder identificar los emojis en cada tweet, donde se busca utilizando EMOJI_DATA y se agregan a una lista, para luego utilizar Counter() para sacar el top 10 de los más usados.

In [100]:
def split_count(text):
    emoji_list = []
    #Encontrar emojis y separarlos
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.EMOJI_DATA for char in word): #Si encuentra un emoji, agregarlo a la lista del tweet
            emoji_list.append(word)

    return emoji_list
@profile
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    df = pd.read_json(file_path, lines=True)
    #Sacar una lista con todos los emojis
    all_emojis = (emoji for sublist in df['content'].apply(split_count) for emoji in sublist)
    #Conteo
    emoji_counts = Counter(all_emojis)
    top_10_emojis = emoji_counts.most_common(10)

    lista = list(top_10_emojis)
    return lista

In [101]:
%mprun -f q2_memory q2_memory(file_path)

ERROR: Could not find file /var/folders/qh/33532dns5z5561hpfw82qfz00000gp/T/ipykernel_38184/1033546099.py



Filename: /usr/local/lib/python3.11/site-packages/memory_profiler.py

Line #    Mem usage    Increment  Occurrences   Line Contents
  1185   4689.6 MiB   4689.6 MiB           1               @wraps(wrapped=func)
  1186                                                     def wrapper(*args, **kwargs):
  1187   4689.6 MiB      0.0 MiB           1                   prof = get_prof()
  1188   1551.6 MiB  -3138.0 MiB           1                   val = prof(func)(*args, **kwargs)
  1189   1551.7 MiB      0.1 MiB           1                   show_results_bound(prof)
  1190   1551.7 MiB      0.0 MiB           1                   return val

Para ahorrar tiempo, se utiliza list comprehension para acelerar el procesamiento de los datos

In [102]:
def split_count(text):
    return [char for char in regex.findall(r'\X', text) if char in emoji.EMOJI_DATA]

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    df = pd.read_json(file_path, lines=True)
    
    emoji_lists = df['content'].apply(split_count)
    all_emojis = [emoji for sublist in emoji_lists for emoji in sublist]
    emoji_counts = Counter(all_emojis)
    top_10_emojis = emoji_counts.most_common(10)

    lista = [(emoji, count) for emoji, count in top_10_emojis]

    return lista
q2_time(file_path)

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1642),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

## Q3

En este caso, se saca los usuarios mencionados mediante la columna mentionedUsers, y como esta tiene estructura de lista de diccionarios, se utiliza una list comprehension para crear una nueva columna que sea los username de cada usuario mencionado por tweet, y luego con Counter() se consigue el top 10

In [103]:
def sacar_usuarios(mentioned_users):
    if mentioned_users is None:
            return []
    return [user['username'] for user in mentioned_users if user and 'username' in user and user['username'] is not None]

@profile
def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    df = pd.read_json(file_path, lines=True)
    df = df[['mentionedUsers']]
    conteo = Counter()

    for usernames_list in df['mentionedUsers']:
        conteo.update(sacar_usuarios(usernames_list))
    return(conteo.most_common(10))


In [104]:
%mprun -f q3_memory q3_memory(file_path)

ERROR: Could not find file /var/folders/qh/33532dns5z5561hpfw82qfz00000gp/T/ipykernel_38184/2297922922.py



Filename: /usr/local/lib/python3.11/site-packages/memory_profiler.py

Line #    Mem usage    Increment  Occurrences   Line Contents
  1185   3418.2 MiB   3418.2 MiB           1               @wraps(wrapped=func)
  1186                                                     def wrapper(*args, **kwargs):
  1187   3418.2 MiB      0.0 MiB           1                   prof = get_prof()
  1188   5435.5 MiB   2017.3 MiB           1                   val = prof(func)(*args, **kwargs)
  1189   5435.5 MiB      0.0 MiB           1                   show_results_bound(prof)
  1190   5435.5 MiB      0.0 MiB           1                   return val

Para ahorrar tiempo se utiliza explode() para evitar crear otras variables, y así acelerar el tiempo de ejecución

In [105]:
def sacar_usuarios(mentioned_users):
    if mentioned_users is None:
            return []
    return [user['username'] for user in mentioned_users if user and 'username' in user and user['username'] is not None]

def q3_time(file_path: str) -> List[Tuple[str, int]]:
    df = pd.read_json(file_path, lines=True)
    df = df[['mentionedUsers']]
    conteo = Counter()

    conteo = Counter(df['mentionedUsers'].apply(sacar_usuarios).explode())

    return(conteo.most_common(10))
q3_time(file_path)

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]